# Prepare Tangelo

In [107]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet
    !pip install qulacs pyscf --quiet

# Molecule definition.
from tangelo import SecondQuantizedMolecule as SQMol

# The minimal import foRr DMET.
from tangelo.problem_decomposition import DMETProblemDecomposition
# Ability to change localization method.
from tangelo.problem_decomposition.dmet import Localization
# Use for VQE ressources estimation vs DMET.
from tangelo.algorithms import VQESolver
# Use for comparison.
from tangelo.algorithms import FCISolver

In [109]:
import numpy as np
from tangelo.toolboxes.ansatz_generator.ansatz import Ansatz
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit
from tangelo.toolboxes.qubit_mappings.mapping_transform import get_qubit_number, fermion_to_qubit_mapping
from tangelo.linq import Circuit, Gate

In [111]:
from tangelo.algorithms.variational import BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver

# Dictionary of resources for each algorithm
algorithm_resources = dict()

In [113]:
# select basis
selected_basis = 'minao'
# select Ansatz
ans = BuiltInAnsatze.UpCCGSD

# VQE

Ground State of LiH

In [117]:
# LiH geometry structure
LiH = [('Li', (0, 0, 0)),('H', (0, 0, 1.5949))]

In [119]:
# Initialize LiH ground state
mol_LiH_g = SQMol(LiH, q=0, spin=0, basis=selected_basis, frozen_orbitals=None, symmetry=True)

options_LiH_g_vqe = {"molecule": mol_LiH_g, "qubit_mapping": "jw", "verbose": False, "ansatz": ans}
vqe_LiH_g = VQESolver(options_LiH_g_vqe)
vqe_LiH_g.build()

# Print the vqe resources
resources_LiH_g_vqe = vqe_LiH_g.get_resources()
print(resources_LiH_g_vqe)

{'qubit_hamiltonian_terms': 118, 'circuit_width': 6, 'circuit_depth': 612, 'circuit_2qubit_gates': 416, 'circuit_var_gates': 72, 'vqe_variational_parameters': 18}


In [121]:
# Ground state energy calculation with VQE, reference values with FCI
vqe_energy_g = vqe_LiH_g.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy_g}")
print(f"CASCI energy = {FCISolver(mol_LiH_g).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_LiH_g.get_resources()


 Ground Singlet state
VQE energy = -7.9798733228389125
CASCI energy = -7.979876345514762


# Now, let's simulate excited state of LiH.

In [123]:
# Initialize first excited state of LiH
mol_LiH_e = SQMol(LiH, q=0, spin=2, basis=selected_basis, frozen_orbitals=None, symmetry=True)

options_LiH_e_vqe = {"molecule": mol_LiH_e, "qubit_mapping": "jw", "verbose": False, "ansatz": ans}
vqe_LiH_e = VQESolver(options_LiH_e_vqe)
vqe_LiH_e.build()

# Print the vqe resources
resources_LiH_e_vqe = vqe_LiH_e.get_resources()
print(resources_LiH_e_vqe)

{'qubit_hamiltonian_terms': 118, 'circuit_width': 6, 'circuit_depth': 612, 'circuit_2qubit_gates': 416, 'circuit_var_gates': 72, 'vqe_variational_parameters': 18}


In [124]:
# Excited state energy calculation with VQE, reference values with FCI
vqe_energy_e = vqe_LiH_e.simulate()
print("\n Lowest Triplet Singlet state")
print(f"VQE energy = {vqe_energy_e}")
print(f"CASCI energy = {FCISolver(mol_LiH_e).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_LiH_e.get_resources()


 Lowest Triplet Singlet state
VQE energy = -7.876815134234935
CASCI energy = -7.876819596419252


# Use VQE_Deflation for first and second excited state energy

In [126]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_LiH_g.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
for i in range(2):
    vqe_options_def = {"molecule": mol_LiH_g, "ansatz": ans, 
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.3, "verbose": False}
    vqe_solver_def = VQESolver(vqe_options_def)
    vqe_solver_def.build()
    vqe_energy_def = vqe_solver_def.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy_def}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver_def.get_resources()

    deflation_circuits.append(vqe_solver_def.optimal_circuit.copy())

Excited state #1 	 VQE energy = -7.752112998108656
Excited state #2 	 VQE energy = -7.679869580679111


In [128]:
# Use a reference state with 2 alpha electrons and 2 beta electrons to calculate the triplet state. 
vqe_options_def2 = {"molecule": mol_LiH_g, "ansatz": ans, 
               "deflation_circuits": deflation_circuits, "deflation_coeff": 0.3, 
               "ref_state": [1, 1, 1, 1]} 

vqe_solver_def2 = VQESolver(vqe_options_def2)
vqe_solver_def2.build()
vqe_energy_def2 = vqe_solver_def2.simulate()
print(f"VQE energy of excited state #1= {vqe_energy_def2}")
algorithm_resources[f"vqe_deflation_state_{3}"] = vqe_solver_def2.get_resources()

VQE energy of excited state #1= -7.876815301741519


In [130]:
from scipy.linalg import eigh
from openfermion.utils import hermitian_conjugated as hc

from tangelo.toolboxes.operators import FermionOperator
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping as f2q_mapping

# Generate all single excitations as qubit operators
op_list = list()
for i in range(2):
    for j in range(i+1, 2):
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j, 0))), "jw")] #spin-up transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j+1, 0))), "jw")] #spin-down transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j, 0))), "jw")] #spin-up to spin-down
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j+1, 0))), "jw")] #spin-down to spin-up

# Compute F and S matrices.
size_mat = len(op_list)
h = np.zeros((size_mat, size_mat))
s = np.zeros((size_mat, size_mat))
state_circuit = vqe_LiH_g.optimal_circuit
for i, op1 in enumerate(op_list):
    for j, op2 in enumerate(op_list):
        h[i, j] = np.real(vqe_LiH_g.backend.get_expectation_value(hc(op1)*vqe_LiH_g.qubit_hamiltonian*op2, state_circuit))
        s[i, j] = np.real(vqe_LiH_g.backend.get_expectation_value(hc(op1)*op2, state_circuit))

label = "quantum_subspace_expansion"
algorithm_resources[label] = vqe_LiH_g.get_resources()
algorithm_resources[label]["n_post_terms"] = len(op_list)**2*algorithm_resources[label]["qubit_hamiltonian_terms"]

In [131]:
# Solve FU = SUE
e, v = eigh(h,s)
print(f"Quantum Subspace Expansion energies for first excited state of LiH: \n {e}")

Quantum Subspace Expansion energies for first excited state of LiH: 
 [-6.86245489 -5.75618619 -5.71042514 -5.66263862]


# DMET-VQE

Decompose LiH into Li and H

In [135]:
# Initialize LiH ground state

mol_LiH_dmet_g = SQMol(LiH, q=0, spin=0, basis='cc-pVDZ', frozen_orbitals=None, symmetry=True)

options_LiH_dmet_g = {"molecule": mol_LiH_dmet_g,
                    "fragment_atoms": [1,1],
                    "fragment_solvers": "ccsd",
                    "verbose": False
                    }

dmet_vqe_LiH_g = DMETProblemDecomposition(options_LiH_dmet_g)
dmet_vqe_LiH_g.build()

In [136]:
fci_LiH_g = FCISolver(mol_LiH_g)
energy_LiH_g_fci = fci_LiH_g.simulate()
print(f"FCI energy (hartree): \t {energy_LiH_g_fci}")

energy_dmet_vqe_LiH_g = dmet_vqe_LiH_g.simulate()
print(f"DMET energy (hartree): \t {energy_dmet_vqe_LiH_g}")

FCI energy (hartree): 	 -7.979876345514762
DMET energy (hartree): 	 -8.0130485999147


In [137]:
energy_LiH_g_hf = dmet_vqe_LiH_g.mean_field.e_tot
energy_corr_LiH_g = abs(energy_dmet_vqe_LiH_g - energy_LiH_g_hf)

print(f"Correlation energy (hartree): \t {energy_corr_LiH_g}")
print(f"Correlation energy (kcal/mol): \t {627.5*energy_corr_LiH_g}")

Correlation energy (hartree): 	 0.02943332512421737
Correlation energy (kcal/mol): 	 18.4694115154464


In [138]:
delta_LiH_g_fci_hf = abs(energy_LiH_g_fci - energy_LiH_g_hf)
delta_LiH_g_fci_dmet = abs(energy_LiH_g_fci - energy_dmet_vqe_LiH_g)

print(f"Difference FCI vs HF energies (hartree): \t\t {delta_LiH_g_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (hartree): \t\t {delta_LiH_g_fci_dmet}")
print(f"Difference FCI vs HF energies (kcal/mol): \t\t {627.5*delta_LiH_g_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (kcal/mol): \t {627.5*delta_LiH_g_fci_dmet}")

Difference FCI vs HF energies (hartree): 		 0.003738929275721503
Difference FCI vs DMET-VQE energies (hartree): 		 0.03317225439993887
Difference FCI vs HF energies (kcal/mol): 		 2.346178120515243
Difference FCI vs DMET-VQE energies (kcal/mol): 	 20.815589635961643


In [139]:
# Initialize first excited state of LiH
mol_LiH_dmet_e = SQMol(LiH, q=0, spin=2, basis='cc-pVDZ', frozen_orbitals=None, symmetry=True)

# Initialize LiH excited state
options_LiH_dmet_e = {"molecule": mol_LiH_dmet_e,
                    "fragment_atoms": [1,1],
                    "fragment_solvers": "ccsd",
                    "verbose": False
                    }

dmet_vqe_LiH_e = DMETProblemDecomposition(options_LiH_dmet_e)
dmet_vqe_LiH_e.build()

In [140]:
fci_LiH_e = FCISolver(mol_LiH_e)
energy_LiH_e_fci = fci_LiH_e.simulate()
print(f"FCI energy (hartree): \t {energy_LiH_e_fci}")

energy_dmet_vqe_LiH_e = dmet_vqe_LiH_e.simulate()
print(f"DMET energy (hartree): \t {energy_dmet_vqe_LiH_e}")

FCI energy (hartree): 	 -7.876819596419252
DMET energy (hartree): 	 -7.900773422217099


In [141]:
energy_LiH_e_hf = dmet_vqe_LiH_e.mean_field.e_tot
energy_corr_LiH_e = abs(energy_dmet_vqe_LiH_e - energy_LiH_e_hf)

print(f"Correlation energy (hartree): \t {energy_corr_LiH_e}")
print(f"Correlation energy (kcal/mol): \t {627.5*energy_corr_LiH_e}")

Correlation energy (hartree): 	 0.0009517907457237129
Correlation energy (kcal/mol): 	 0.5972486929416299


In [142]:
delta_LiH_e_fci_hf = abs(energy_LiH_e_fci - energy_LiH_e_hf)
delta_LiH_e_fci_dmet = abs(energy_LiH_e_fci - energy_dmet_vqe_LiH_e)

print(f"Difference FCI vs HF energies (hartree): \t\t {delta_LiH_e_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (hartree): \t\t {delta_LiH_e_fci_dmet}")
print(f"Difference FCI vs HF energies (kcal/mol): \t\t {627.5*delta_LiH_e_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (kcal/mol): \t {627.5*delta_LiH_e_fci_dmet}")

Difference FCI vs HF energies (hartree): 		 0.02300203505212295
Difference FCI vs DMET-VQE energies (hartree): 		 0.023953825797846662
Difference FCI vs HF energies (kcal/mol): 		 14.43377699520715
Difference FCI vs DMET-VQE energies (kcal/mol): 	 15.03102568814878


Compute LiH with other ansatz

In [144]:
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit
from tangelo.toolboxes.qubit_mappings.mapping_transform import get_qubit_number, fermion_to_qubit_mapping
from tangelo.linq import Circuit, Gate

In [145]:
def EulerCircuit(n_qubits):
    """Construct a circuit applying an Euler Z-X-Z rotation to each qubit."""
    circuit = Circuit()
    for target in range(n_qubits):
        circuit.add_gate(Gate("RZ" , target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RX", target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RZ", target, parameter=0.0, is_variational=True))
    return circuit

def EntanglerCircuit(n_qubits):
    """Construct a circuit applying two columns of staggered CNOT gates to all qubits
     and their neighbours"""
    circuit = Circuit()
    for ii in range(n_qubits//2):
        circuit.add_gate(Gate("CNOT", control=2*ii, target=2*ii + 1))
    for ii in range(n_qubits//2 - 1):
        circuit.add_gate(Gate("CNOT", control=2*ii + 1, target=2*(ii+1)))
    return circuit

def HEACircuit(n_qubits, n_layers):
    """Construct a circuit consisting of alternating sequence of Euler rotations and entanglers"""
    circuit = EulerCircuit(n_qubits)
    for ii in range(n_layers):
        circuit += EntanglerCircuit(n_qubits)
        circuit += EulerCircuit(n_qubits)
    return circuit

In [146]:
class HEA(Ansatz):

    def __init__(self, n_spinorbitals, n_electrons, n_layers, mapping='jw'):

        self.n_spinorbitals = n_spinorbitals
        self.n_qubits = get_qubit_number(mapping, n_spinorbitals)
        self.n_electrons = n_electrons
        #number of layers of repeated entangler + Euler rotations
        self.n_layers = n_layers
        
        #specify fermion-to-qubit mapping (required for the initial reference state)
        self.mapping = mapping
        
        #Each layer has 3 variational parameters per qubit, and one non-variational entangler
        #There is an additional layer with no entangler.
        self.n_var_params = self.n_qubits * 3 * (self.n_layers + 1)

        self.var_params = None
        self.circuit = None

    def set_var_params(self, var_params=None):
        """Set initial variational parameter values"""
        if var_params is None:
            var_params = np.random.random(self.n_var_params)
        elif isinstance(var_params, str) and var_params == "ones":
            var_params = np.ones(self.n_var_params, dtype=float)
        elif len(var_params) != self.n_var_params:
            raise ValueError('Invalid number of parameters.')
        self.var_params = var_params
        return var_params

    def update_var_params(self, var_params):
        """Update variational parameters (done repeatedly during VQE)"""
        for param_index in range(self.n_var_params):
            self.circuit._variational_gates[param_index].parameter = var_params[param_index]
    
    def prepare_reference_state(self):
        """Prepare a circuit generating the HF reference state."""
        return get_reference_circuit(n_spinorbitals=self.n_spinorbitals, n_electrons=self.n_electrons,mapping=self.mapping)

    def build_circuit(self, var_params=None):
        """Construct the variational circuit to be used as our ansatz."""
        self.var_params = self.set_var_params(var_params)

        reference_state_circuit = self.prepare_reference_state()
        hea_circuit = HEACircuit(self.n_qubits, self.n_layers)

        if reference_state_circuit.size != 0:
            self.circuit = reference_state_circuit + hea_circuit
        else:
            self.circuit = hea_circuit
        return self.circuit

In [147]:
n_spinorbitals = mol_LiH_g.n_active_sos
n_electrons = mol_LiH_g.n_active_electrons
hea_layers = 4
HEA_ansatz = HEA(n_spinorbitals=n_spinorbitals, n_electrons=n_electrons, n_layers=hea_layers)

options_LiH_g_h_vqe = {"molecule": mol_LiH_g, "qubit_mapping": "jw", "verbose": False, "ansatz": HEA_ansatz}
vqe_LiH_g_h = VQESolver(options_LiH_g_h_vqe)
vqe_LiH_g_h.build()

# Print the vqe resources
resources_LiH_g_h_vqe = vqe_LiH_g_h.get_resources()
print(resources_LiH_g_h_vqe)

# Ground state energy calculation with VQE, reference values with FCI
vqe_energy_g = vqe_LiH_g_h.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy_g}")
print(f"CASCI energy = {FCISolver(mol_LiH_g).simulate()}")

{'qubit_hamiltonian_terms': 118, 'circuit_width': 6, 'circuit_depth': 24, 'circuit_2qubit_gates': 20, 'circuit_var_gates': 90, 'vqe_variational_parameters': 90}

 Ground Singlet state
VQE energy = -7.979693574432048
CASCI energy = -7.979876345514762


In [148]:
n_spinorbitals = mol_LiH_e.n_active_sos
n_electrons = mol_LiH_e.n_active_electrons
hea_layers = 4
HEA_ansatz = HEA(n_spinorbitals=n_spinorbitals, n_electrons=n_electrons, n_layers=hea_layers)

options_LiH_e_h_vqe = {"molecule": mol_LiH_e, "qubit_mapping": "jw", "verbose": False, "ansatz": HEA_ansatz}
vqe_LiH_e_h = VQESolver(options_LiH_e_h_vqe)
vqe_LiH_e_h.build()

# Print the vqe resources
resources_LiH_e_h_vqe = vqe_LiH_e_h.get_resources()
print(resources_LiH_e_h_vqe)

# Ground state energy calculation with VQE, reference values with FCI
vqe_energy_e = vqe_LiH_e_h.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy_e}")
print(f"CASCI energy = {FCISolver(mol_LiH_e).simulate()}")

{'qubit_hamiltonian_terms': 118, 'circuit_width': 6, 'circuit_depth': 24, 'circuit_2qubit_gates': 20, 'circuit_var_gates': 90, 'vqe_variational_parameters': 90}

 Ground Singlet state
VQE energy = -7.978741471622649
CASCI energy = -7.876819596419252


In [149]:
# Define the H2O element
H2O = [('H', (0, 0, 0)), ('O', (0.758, 0.0, 0.586)), ('O', (-0.758, 0.0, 0.586))]

# Create a SecondQuantizedMolecule instance for H2O
mol_H2O = SQMol(H2O, q=0, spin=1, basis=selected_basis, frozen_orbitals=6)

n_spinorbitals = mol_H2O.n_active_sos
n_electrons = mol_H2O.n_active_electrons
hea_layers = 4
HEA_ansatz = HEA(n_spinorbitals=n_spinorbitals, n_electrons=n_electrons, n_layers=hea_layers)

options_H2O_vqe = {"molecule": mol_H2O, "qubit_mapping": "jw", "verbose": False, "ansatz": HEA_ansatz}
vqe_H2O = VQESolver(options_H2O_vqe)
vqe_H2O.build()

# Print the vqe resources
resources_H2O_vqe = vqe_H2O.get_resources()
print(resources_H2O_vqe)

# Ground state energy calculation with VQE, reference values with FCI
vqe_energy = vqe_H2O.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_H2O).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_H2O.get_resources()

{'qubit_hamiltonian_terms': 252, 'circuit_width': 10, 'circuit_depth': 24, 'circuit_2qubit_gates': 36, 'circuit_var_gates': 150, 'vqe_variational_parameters': 150}

 Ground Singlet state
VQE energy = -149.83641635822607
CASCI energy = -149.90440227772805
